In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.filterwarnings("ignore")

from GradCAM import *
from define_model import *
from load_label import *
from utilities import *
from Parse_TFrecords import *
import gc
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import numpy as np
from sklearn.metrics import classification_report, average_precision_score

gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
print(gpus)
 
print(tf.__version__)

In [ ]:
BATCH_SIZE = 32
dataset = 'emory'

record_file_test = 'tfrecords/copd_{a}_test.tfrecords'.format(a=dataset)
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test))

y_test, y_demo = get_data_label(dataset=dataset, split='test', return_demo=True)

In [ ]:
checkpoint_filepath = 'checkpoints_mimic/checkpoint_BCE_DenseNet121'
archi = 'DenseNet121'
model = define_model(archi)
model.load_weights(checkpoint_filepath)

# For GradCam

# last_conv_layer_name = 'relu'
# classifier_layer_names = ['max_pool', 'dense']

# # First, we create a model that maps the input image to the activations
# # of the last conv layer
# last_conv_layer = model.get_layer(last_conv_layer_name)
# last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

# # Second, we create a model that maps the activations of the last conv
# # layer to the final class predictions
# classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
# x = classifier_input
# for layer_name in classifier_layer_names:
#     x = model.get_layer(layer_name)(x)
# classifier_model = tf.keras.Model(classifier_input, x)

# For GradCam++

conv_layer = model.get_layer("relu")
heatmap_model = tf.keras.Model([model.inputs], [conv_layer.output, model.output])

In [ ]:
heat_map = []
preds = []
img = []
ids = []
for idx, i in enumerate(test_dataset):
    if (y_test[idx] == 1):
        img_array = np.reshape(i, (1, 256, 256, 3))
        pred = model.predict(img_array)
        if(pred >= 0.9):
            img.append(img_array)
#             heat_map.append(show_heatmap(img_array, last_conv_layer_model, classifier_model))
            heat_map.append(grad_cam_plus(heatmap_model, img_array))
            preds.append(pred)
            ids.append(idx)
                        
preds = np.array(preds).reshape(len(preds,))
ids = np.array(ids).reshape(len(ids,))
order = np.argsort(np.array(preds))

In [ ]:
for i in order[-20:]:
    print(preds[i])
    plt.imshow(np.reshape(heat_map[i], (256, 256, 3)))
    fname = 'imgs/imgs_mimic/external/{a}_{b}.jpg'.format(a=ids[i], b=preds[i])
    plt.savefig(fname)
    plt.show()

In [ ]:
mean_heatmap = np.mean(heat_map, axis=0)
plt.imshow(np.reshape(mean_heatmap, (256, 256, 3)))
fname = 'imgs/imgs_mimic/external/mean.jpg'.format(a=ids[i], b=preds[i])
plt.savefig(fname)
plt.show()

## Saliency Map

In [ ]:
def get_saliency_map(model, image):
    with tf.GradientTape() as tape:
        tape.watch(image)
        loss = model(image)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, image)

    dgrad_abs = tf.math.abs(gradient)
#         print(dgrad_abs.shape)
    dgrad_max_ = np.max(dgrad_abs, axis=-1)[0]
#         print(dgrad_max_.shape)

    # normaliz between 0 and 1
    arr_min, arr_max  = np.min(dgrad_max_), np.max(dgrad_max_)
    smap = (dgrad_max_ - arr_min) / (arr_max - arr_min + 1e-18)

    return smap

In [ ]:
linear_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-1].output)
for i in order[-20:]:
    print(preds[i])
    smap = get_saliency_map(linear_model, tf.Variable(img[i], dtype=float))

    plt.subplot(1, 2, 1)
    plt.imshow(img[i][0])

    plt.subplot(1, 2, 2)
    plt.imshow(smap, cmap='Reds')

    plt.show()

In [ ]:
smaps = []
for i in range(len(img)):
    smaps.append(get_saliency_map(linear_model, tf.Variable(img[i], dtype=float)))

In [ ]:
mean_img = np.mean(img, axis=0)
plt.subplot(1, 2, 1)
plt.imshow(mean_img[0], cmap='gray')

mean_smap = np.mean(smaps, axis=0)
plt.subplot(1, 2, 2)
plt.imshow(mean_smap, cmap='Reds')

plt.show()